In [4]:
import tensorflow as tf
import numpy as np

In [5]:
tf.executing_eagerly()

True

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = np.array(y_train, dtype=np.int32)

11493376/11490434 [==============================] - 0s 0us/step


In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [39]:
optimizer = tf.keras.optimizers.Adam()

In [79]:
@tf.function
def compute_loss_train(minibatch_x, minibatch_y):
    with tf.GradientTape() as tape:
        y_pred = model(minibatch_x)
        loss_obj = tf.nn.sparse_softmax_cross_entropy_with_logits(minibatch_y, y_pred)
        loss = tf.reduce_mean(loss_obj)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [88]:
def train_minibatch(minibatch_size=128):
    n_minibatch = len(x_train) // minibatch_size + 1
    losses = []
    for b in range(n_minibatch):
        mb_x = x_train[b * minibatch_size:(b + 1) * minibatch_size]
        mb_y = y_train[b * minibatch_size:(b + 1) * minibatch_size]
        loss = compute_loss_train(mb_x, mb_y).numpy()
        losses.append(loss)
    mean_loss = np.mean(losses)
    return mean_loss

In [90]:
%timeit train_minibatch()

1.92 s ± 8.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [97]:
model.compile('adam', tf.keras.losses.SparseCategoricalCrossentropy())

In [99]:
model.fit(x_train, y_train, batch_size=128, verbose=2, epochs=20)s

Epoch 1/20
469/469 - 2s - loss: 1.9704
Epoch 2/20
469/469 - 2s - loss: 2.0841
Epoch 3/20
469/469 - 2s - loss: 2.2076
Epoch 4/20
469/469 - 2s - loss: 2.0805
Epoch 5/20
469/469 - 2s - loss: 2.0752
Epoch 6/20
469/469 - 2s - loss: 2.2377
Epoch 7/20
469/469 - 2s - loss: 2.2220
Epoch 8/20
469/469 - 2s - loss: 2.2223
Epoch 9/20
469/469 - 2s - loss: 2.2812
Epoch 10/20
469/469 - 2s - loss: 2.2920
Epoch 11/20
469/469 - 2s - loss: 2.3026
Epoch 12/20
469/469 - 2s - loss: 2.3026
Epoch 13/20
469/469 - 2s - loss: 2.3025
Epoch 14/20
469/469 - 2s - loss: 2.3025
Epoch 15/20
469/469 - 2s - loss: 2.3025
Epoch 16/20
469/469 - 2s - loss: 2.3026
Epoch 17/20
469/469 - 2s - loss: 2.3025
Epoch 18/20
469/469 - 2s - loss: 2.3024
Epoch 19/20
469/469 - 2s - loss: 2.3024
Epoch 20/20
469/469 - 2s - loss: 2.3024


In [56]:
%timeit compute_loss_train()

74.6 ms ± 4.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
tf.function(compute_loss_train)()

<tf.Tensor: shape=(), dtype=float32, numpy=0.22579737>

In [54]:
model(x_train)

<tf.Tensor: shape=(60000, 10), dtype=float32, numpy=
array([[ 1.346983  , -5.256694  ,  1.4213378 , ..., -0.16308315,
         1.9537978 , -2.9795372 ],
       [11.373805  , -5.1438828 , -1.1088789 , ..., -4.229908  ,
        -0.93692875, -4.211445  ],
       [-2.1506717 , -6.1678457 , -3.1803238 , ...,  2.6911044 ,
        -0.9427343 ,  4.453407  ],
       ...,
       [-0.9134552 , -1.4624233 , -2.8580995 , ..., -2.3208833 ,
         2.3583336 , -0.80597425],
       [ 2.6301992 , -4.5024195 , -0.09712442, ..., -5.233919  ,
        -2.006183  , -2.9430153 ],
       [-0.38782105, -1.2763739 ,  1.1522961 , ..., -0.48857155,
         3.4760864 , -0.46510482]], dtype=float32)>